# Random Forest Regression of the defence of NFL teams through the 2018-21 seasons 


All modules and libraries imported. csv containing raw data from ProFootballReference.com is also imported and read by the pandas read function. This is defined
as df.

In [46]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.graphics.factorplots import interaction_plot
from scipy import stats
import matplotlib.colors as mcolors
from scipy.stats import rankdata
import seaborn as sns
df = pd.read_csv (r'C:\Users\Rob\Documents\dstats.csv')
print (df)

                        Tm   G  Cmp  Att  Cmp%   Yds  TD  TD%  Int  PD  ...  \
0        Arizona Cardinals  17  367  561  65.4  3645  30  5.3   13  73  ...   
1          Atlanta Falcons  17  391  577  67.8  3952  31  5.4   12  77  ...   
2         Baltimore Ravens  17  397  621  63.9  4742  31  5.0    9  72  ...   
3            Buffalo Bills  17  297  530  56.0  2771  12  2.3   19  80  ...   
4        Carolina Panthers  17  337  515  65.4  3266  26  5.0    9  52  ...   
..                     ...  ..  ...  ...   ...   ...  ..  ...  ...  ..  ...   
91   San Francisco 49ers19  16  318  519  61.3  2707  23  4.4   12  75  ...   
92      Seattle Seahawks19  16  383  598  64.0  4223  19  3.2   16  74  ...   
93  Tampa Bay Buccaneers19  16  408  664  61.4  4322  30  4.5   12  96  ...   
94      Tennessee Titans19  16  386  598  64.5  4080  25  4.2   14  72  ...   
95   Washington Redskins19  16  371  540  68.7  3823  35  6.5   13  52  ...   

    Hrry   Hrry%  QBKD   QBKD%  aSk  Prss   Prss%  

Finding the mean of all points allowed and added back to the df to use as the baseline later

In [47]:
np.mean(df['PA'])


384.1041666666667

Checking that all the cells in the dataframe is filled

In [48]:
df.describe()

,G,Cmp,Att,Cmp%,Yds,TD,TD%,Int,PD,Int%,...,Air,aYAC,Bltz,Hrry,QBKD,aSk,Prss,MTkl,PA,PAA
count,96.000000,96.000000,96.000000,96.000000,96.000000,96.00000,96.000000,96.000000,96.000000,96.000000,...,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,9.600000e+01
mean,16.333333,366.750000,568.572917,64.473958,3827.718750,26.12500,4.606250,12.968750,68.968750,2.288542,...,2294.708333,1933.708333,178.468750,62.020833,49.697917,38.072917,149.791667,107.989583,384.104167,3.841000e+02
std,0.473879,33.964688,41.337060,3.149812,428.987614,5.55783,0.991417,4.027414,11.484042,0.720507,...,307.854850,276.118202,50.766328,13.440692,11.606822,9.128607,23.402167,16.409557,59.334463,7.428436e-13
min,16.000000,297.000000,464.000000,56.000000,2707.000000,12.00000,2.300000,3.000000,43.000000,0.600000,...,1707.000000,1354.000000,72.000000,35.000000,27.000000,17.000000,95.000000,66.000000,225.000000,3.841000e+02
25%,16.000000,343.750000,541.000000,62.675000,3577.500000,22.00000,3.900000,10.000000,60.750000,1.800000,...,2108.000000,1734.250000,143.750000,52.000000,41.000000,31.750000,135.500000,96.750000,351.000000,3.841000e+02
50%,16.000000,368.000000,562.000000,64.050000,3822.000000,26.00000,4.600000,12.500000,70.000000,2.250000,...,2258.000000,1906.000000,168.500000,59.500000,50.000000,39.000000,148.000000,109.000000,373.000000,3.841000e+02
75%,17.000000,390.250000,598.000000,66.650000,4123.000000,30.00000,5.225000,15.000000,77.250000,2.625000,...,2487.250000,2071.500000,206.750000,71.000000,56.000000,46.000000,165.250000,119.000000,426.000000,3.841000e+02
max,17.000000,450.000000,680.000000,70.700000,4742.000000,39.00000,7.200000,26.000000,96.000000,4.700000,...,3118.000000,2793.000000,329.000000,95.000000,80.000000,56.000000,219.000000,143.000000,519.000000,3.841000e+02


One-hot Encoding to turn the Categorical data into numbers to allow better analysis through the machine learning

In [49]:
df = pd.get_dummies(df)
df.iloc[:,5:].head(5)

,TD,TD%,Int,PD,Int%,Y/A,AY/A,Y/C,Y/G,Rate,...,Prss%_26.80%,Prss%_27.50%,Prss%_27.60%,Prss%_27.90%,Prss%_28.50%,Prss%_28.60%,Prss%_28.70%,Prss%_30.50%,Prss%_30.80%,Prss%_35.10%
0,30,5.3,13,73,2.3,6.9,6.9,10.6,214.4,93.5,...,0,0,0,0,0,0,0,0,0,0
1,31,5.4,12,77,2.1,7.1,7.3,10.5,232.5,97.4,...,0,0,0,0,0,0,0,0,0,0
2,31,5.0,9,72,1.4,8.0,8.4,12.6,278.9,99.4,...,0,0,0,0,0,0,0,0,0,0
3,12,2.3,19,80,3.6,5.7,4.6,10.2,163.0,65.3,...,0,0,0,0,0,0,0,0,1,0
4,26,5.0,9,52,1.7,6.9,7.1,10.6,192.1,95.0,...,0,0,0,0,0,0,0,0,0,0


### Choose what is to be predicted

Define the desired variable to model for (predict) as labels. Then remove that column from the dataframe and return the dataframe as an array.

In [50]:
labels = np.array(df['PA'])
df= df.drop('PA', axis = 1)
df_list = list(df.columns)
df = np.array(df)

Import the train_test_split function from sklearn

In [51]:
from sklearn.model_selection import train_test_split

Create the training and testing splits. Also add the size of the testing data.

In [52]:
train_df, test_df, train_labels, test_labels = train_test_split(df, labels, test_size = 0.25, random_state = 42)

Print the shapes of the splits created above to make sure there are no errors in them


In [53]:
print('Training df Shape:', train_df.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing df Shape:', test_df.shape)
print('Testing Labels Shape:', test_labels.shape)

Training df Shape: (72, 400)
Training Labels Shape: (72,)
Testing df Shape: (24, 400)
Testing Labels Shape: (24,)


### Baseline

Use the PAA (Points Against Average) created earlier as the baseline prediction. Test for errors in the baseline compared to the testing data

In [54]:
baseline_preds = test_df[:, df_list.index('PAA')]
baseline_errors = abs(baseline_preds - test_labels)
print('Average baseline error: ', round(np.mean(baseline_errors), 2))

Average baseline error:  53.93


### Random Forest Regression

Import the RandomForestRegression from Sklearn. Then call for the number of decision trees required, in this case 1000. Now train the model to the training set created.

In [55]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(train_df, train_labels);

Now make the predictions based on the testing data and calculate the mean absolute error

In [56]:
predictions = rf.predict(test_df)
errors = abs(predictions - test_labels)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 27.58 degrees.


Now find the accuracy of the predictions by calculating the mean absolute error percentage and subtracting the mean of the mape from 100. 

In [57]:
mape = 100 * (errors / test_labels)
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 92.56 %.


### List of Importances

Make a list of the variables and their importance in the prediction. The list is then sorted from most important to least. 

In [59]:
importances = list(rf.feature_importances_)
feature_importances = [(df, round(importance, 2)) for df, importance in zip(df_list, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: ANY/A                Importance: 0.24
Variable: Rate                 Importance: 0.19
Variable: EXP                  Importance: 0.09
Variable: AY/A                 Importance: 0.06
Variable: NY/A                 Importance: 0.04
Variable: TD                   Importance: 0.03
Variable: Attr                 Importance: 0.03
Variable: Ydsr                 Importance: 0.03
Variable: TDr                  Importance: 0.03
Variable: RY/G                 Importance: 0.03
Variable: aTD                  Importance: 0.03
Variable: Cmp                  Importance: 0.01
Variable: Cmp%                 Importance: 0.01
Variable: Yds                  Importance: 0.01
Variable: TD%                  Importance: 0.01
Variable: Y/G                  Importance: 0.01
Variable: Yds.1                Importance: 0.01
Variable: QBHits               Importance: 0.01
Variable: REXP                 Importance: 0.01
Variable: aCmp                 Importance: 0.01
Variable: aYds                 Importanc